<a href="https://colab.research.google.com/github/bhumong/ai-bootcamp/blob/main/project/03B_pytorch_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Deep Learning - Pytorch

**Instructions for Students:**

Please carefully follow these steps to complete and submit your project:

1. **Make a copy of the Project**: Please make a copy of this project either to your own Google Drive or download locally. Work on the copy of the project. The master project is **Read-Only**, meaning you can edit, but it will not be saved when you close the master project. To avoid total loss of your work, remember to make a copy.

2. **Completing the Project**: You are required to work on and complete all tasks in the provided project. Be disciplined and ensure that you thoroughly engage with each task.
   
3. **Creating a Google Drive Folder**: Each of you must create a new folder on your Google Drive. This will be the repository for all your completed project files, aiding you in keeping your work organized and accessible.
   
4. **Uploading Completed Project**: Upon completion of your project, make sure to upload all necessary files, involving codes, reports, and related documents into the created Google Drive folder. Save this link in the 'Student Identity' section and also provide it as the last parameter in the `submit` function that has been provided.
   
5. **Sharing Folder Link**: You're required to share the link to your project Google Drive folder. This is crucial for the submission and evaluation of your project.
   
6. **Setting Permission to Public**: Please make sure your Google Drive folder is set to public. This allows your instructor to access your solutions and assess your work correctly.

Adhering to these procedures will facilitate a smooth project evaluation process for you and the reviewers.

## Project Description

The Deep Learning Projects are divided into two parts, the first is the Calculations worth 30% and the second one is Pytorch Project worth 70% in this notebook.

The two projects will help you gain experience to learn about Deep Learning in detail.

In this project, you will use what you learn to create your own Deep Learning model. We'll use a variety of datasets, each with different data types such as images, text, and numerical/categorical data. Your task is to choose one of these datasets and build a deep learning model upon it.


## Datasets

Choose one from the following datasets for your project:

1. **Digits Dataset**: A simplified version of MNIST containing 8x8 images of hand-written digits. [Dataset Link](https://scikit-learn.org/stable/auto_examples/datasets/plot_digits_last_image.html)
2. **Dogs vs. Cats Dataset from Kaggle**: A sizable dataset of 25,000 images with equal representation of dogs and cats. [Dataset Link](https://www.kaggle.com/c/dogs-vs-cats)
3. **Breast Cancer Wisconsin (Diagnostic) Dataset**: Computed features from a digitized image of a fine needle aspirate (FNA) of a breast mass. [Dataset Link](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)
4. **Spam Text Message Classification Dataset**: A collection of labeled SMS messages, categorized as "spam" or "ham". [Dataset Link](https://www.kaggle.com/uciml/sms-spam-collection-dataset)
5. **German Credit Risk Dataset**: People's data categorized by various attributes to predict credit risk. [Dataset Link](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29)



## Grading Criteria

Your work will be evaluated based on both accuracy and loss value:

100: The model has an accuracy of more than 80% and a Loss Value of less than 0.2. This model is excellent and demonstrates a strong understanding of the task.

90: The model has an accuracy between 70% - 79% and a Loss Value between 0.2 - 0.3. This model is very good, with some room for improvement.

80: The model has an accuracy between 60% - 69% and a Loss Value between 0.3 - 0.4. This model is fairly good but needs improvement in balancing accuracy and loss value.

70: The model has an accuracy between 50% - 59% and a Loss Value between 0.4 - 0.5. This model is below average and needs significant improvement.

60 or below: The model has an accuracy of less than 50% or a Loss Value of more than 0.5, or the student did not submit the accuracy and Loss Value. This model is poor and needs considerable improvement.

Rmember to make a copy of this notebook in your Google Drive and work in your own copy.

Let's start your deep learning journey! Choose your dataset and delve into the project! Happy modeling!

## Student Identity

In [ ]:
# @title #### Student Identity
student_id = "" # @param {type:"string"}
name = "" # @param {type:"string"}
drive_link = ""  # @param {type:"string"}

assignment_id = "00_pytorch_project"

# Import grader package
!pip install rggrader
from rggrader import submit, submit_image

## Project Structure

Your project should be organized into five main sections.

### 1. Package and Module Installation

First, let's pool all package and module that you'll need in the installation section below.

In [ ]:
!kaggle datasets download uciml/sms-spam-collection-dataset

In [ ]:
!unzip sms-spam-collection-dataset.zip

In [ ]:
!pip install pandas torch scikit-learn

### 2. Data Loading and Preprocessing

Load the chosen dataset and preprocess it for deep learning.

In [ ]:
# Write your preprocessing here
import pandas as pd

data = pd.read_csv("./spam.csv", header=None, encoding='latin-1')
data = data[[0, 1]]
data.columns = ['label', 'text']
data.drop(0, inplace= True)

data['label'] = data['label'].map({"ham": 0, "spam": 1})
data

In [ ]:
def tokenize(text):
    return text.lower().split()

data['tokens'] = data['text'].apply(tokenize)
data

In [ ]:

from collections import Counter

counter = Counter()
for tokens in data['tokens']:
    counter.update(tokens)
counter

In [ ]:
vocab = {'<PAD>': 0, '<UNK>': 1}
for word, freq in counter.items():
    if word not in vocab:
        vocab[word] = len(vocab)
vocab

In [ ]:
def tokens_to_indices(tokens, vocab):
    return [vocab.get(token, vocab['<UNK>']) for token in tokens]
data['indices'] = data['tokens'].apply(lambda tokens: tokens_to_indices(tokens, vocab))
data

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(len(train_data), len(test_data))

In [ ]:
import torch

class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df.copy()
        self.df.reset_index(drop=True, inplace=True)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        indices = self.df.loc[index, 'indices']
        label = self.df.loc[index, 'label']
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

train_dataset = SpamDataset(train_data)
test_dataset = SpamDataset(test_data)
print(train_dataset, test_dataset)

In [ ]:
from torch.utils.data import DataLoader

def collate_batch(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=vocab['<PAD>'])
    labels = torch.stack(labels)
    return texts_padded, labels

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_batch)


### 3. Model Building

Define your deep learning model's architecture.

In [ ]:
# Write your code here for Model Building here

# define your Deep Learning Model here, training is in the next section.

from torch import nn
import logging
class SpamDetection(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, padding_idx):
        super(SpamDetection, self).__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        # print({"embedded": embedded})

        lstm_out, (h_n, c_n) = self.lstm(embedded)
        # print({"lstm_out": lstm_out, "h_n": h_n, "c_n": c_n})

        out = self.fc(h_n[-1])
        # print({"out": out})
        return out

### 4. Model Training

Train your model and evaluate its performance using validation data.

In [ ]:
# Write your code here for Model Training here
dl_loss_value = 1

#define the iteration
import torch.optim as optim
from torch import nn

vocab_size = len(vocab)
embed_dim = 100
hidden_dim = 128
num_classes = 2
padding_idx = vocab['<PAD>']

model = SpamDetection(vocab_size, embed_dim, hidden_dim, num_classes, padding_idx)
device = "cpu"
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#create the training loop
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import confusion_matrix, classification_report

latest_pred = {"accuracy": None, "cm": None, "report": None}
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    dl_loss_value = avg_loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    all_labels = []
    all_preds = []
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=['ham', 'spam'])

    latest_pred["accuracy"] = accuracy
    latest_pred["cm"] = cm
    latest_pred["report"] = report


print("Training complete!")


### 5. Model Evaluation
Evaluate your model's performance on the test data using the grading scheme defined above.

In [ ]:
print(f"accuracy: {latest_pred['accuracy']}")

print("confusion matrix:")
print(latest_pred['cm'])

print("classification report:")
print(latest_pred['report'])

# Write your code here for Model Evaluation here
dl_accuracy = latest_pred['accuracy']

#define the iteration

#create the training loop


## Submission

Once you are satisfied with the performance of your model, then you run the code block below to submit your project.


In [ ]:
# Submit Method

# Do not change the code below
question_id = "01_dl_project_accuracy"
submit(student_id, name, assignment_id, str(dl_accuracy), question_id, drive_link)
question_id = "02_dl_project_loss_value"
submit(student_id, name, assignment_id, str(dl_loss_value), question_id, drive_link)

## FIN